In [ ]:
# Databricks notebook source
# =============================================================================
# 🚀 PRODUCTION SERVING ENDPOINT — SINGLE MODEL ARCHITECTURE
# =============================================================================

from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import (
    EndpointCoreConfigInput,
    ServedEntityInput
)
import mlflow
from mlflow.tracking import MlflowClient
import time, sys, os

print("=" * 80)
print("🚀 STARTING PRODUCTION SERVING ENDPOINT DEPLOYMENT")
print("=" * 80)


# =============================================================================
# ✅ FIXED CONFIG (MUST MATCH TRAINING + REGISTER + PROMOTION SCRIPTS)
# =============================================================================
UC_CATALOG = "workspace"
UC_SCHEMA = "ml"

MODEL_NAME = f"{UC_CATALOG}.{UC_SCHEMA}.house_price_xgboost_uc2"
ENDPOINT_NAME = "house-price-xgboost-prod"

PRODUCTION_ALIAS = "production"

WORKLOAD_SIZE = "Small"
SCALE_TO_ZERO = True


# =============================================================================
# ✅ CLIENTS INITIALIZATION
# =============================================================================
try:
    w = WorkspaceClient()
    mlflow.set_registry_uri("databricks-uc")
    client = MlflowClient()

    print("✅ WorkspaceClient initialized")
    print("✅ MLflow client initialized\n")

except Exception as e:
    print("❌ Failed to initialize clients:", e)
    sys.exit(1)


# =============================================================================
# ✅ STEP 1: Fetch latest PRODUCTION model version
# =============================================================================
def get_production_version():
    print("🔍 Fetching latest PRODUCTION version...")

    versions = client.search_model_versions(f"name='{MODEL_NAME}'")

    for v in versions:
        mv = client.get_model_version(MODEL_NAME, v.version)
        if PRODUCTION_ALIAS in mv.aliases:
            print(f"✅ Production Model Version: {mv.version}")
            print(f"   Run ID: {mv.run_id}\n")
            return mv.version

    print("❌ No production model version found. Run promotion job first.")
    return None


# =============================================================================
# ✅ STEP 2: Check if endpoint exists
# =============================================================================
def endpoint_exists(endpoint_name):
    print(f"🔍 Checking if endpoint '{endpoint_name}' exists...")

    try:
        eps = w.serving_endpoints.list()

        for ep in eps:
            if ep.name == endpoint_name:
                print("✅ Endpoint exists\n")
                return True

        print("ℹ️ Endpoint does NOT exist\n")
        return False

    except Exception as e:
        print("⚠️ Error checking endpoint:", e)
        return False


# =============================================================================
# ✅ STEP 3: Deploy (create/update)
# =============================================================================
def deploy_endpoint(endpoint_name, model_name, version, exists):

    served = ServedEntityInput(
        entity_name=model_name,
        entity_version=version,
        workload_size=WORKLOAD_SIZE,
        scale_to_zero_enabled=SCALE_TO_ZERO
    )

    print("=" * 80)
    print("📦 DEPLOYMENT CONFIGURATION")
    print("=" * 80)
    print(f"Endpoint Name: {endpoint_name}")
    print(f"Model Name:    {model_name}")
    print(f"Model Version: {version}")
    print(f"Workload Size: {WORKLOAD_SIZE}")
    print(f"Scale-to-zero: {SCALE_TO_ZERO}\n")

    try:
        if exists:
            print("🔄 Updating existing endpoint...")
            w.serving_endpoints.update_config(
                name=endpoint_name,
                served_entities=[served]
            )
            print("✅ Update triggered\n")

        else:
            print("🆕 Creating new endpoint...")
            cfg = EndpointCoreConfigInput(
                name=endpoint_name,
                served_entities=[served]
            )
            w.serving_endpoints.create(
                name=endpoint_name,
                config=cfg
            )
            print("✅ Creation triggered\n")

        return True

    except Exception as e:
        print("❌ Deployment Failed:", e)
        return False


# =============================================================================
# ✅ STEP 4: Wait until endpoint is ready
# =============================================================================
def wait_until_ready(endpoint_name):

    print("⏳ Waiting for endpoint to become READY...\n")

    timeout = 1500
    step = 15
    waited = 0

    while waited < timeout:

        ep = w.serving_endpoints.get(name=endpoint_name)
        state = ep.state

        update_state = str(state.config_update)
        ready_state = str(state.ready)

        if "NOT_UPDATING" in update_state and "READY" in ready_state:
            print("\n✅ Endpoint READY for production traffic!\n")
            return True

        if "FAILED" in update_state:
            print("❌ Endpoint FAILED:", update_state)
            return False

        print(f"⏳ {waited}s | Update: {update_state} | Ready: {ready_state}")
        time.sleep(step)
        waited += step

    print("❌ Timeout — endpoint not ready")
    return False


# =============================================================================
# ✅ MAIN
# =============================================================================
if __name__ == "__main__":

    version = get_production_version()
    if not version:
        sys.exit(1)

    exists = endpoint_exists(ENDPOINT_NAME)

    if not deploy_endpoint(ENDPOINT_NAME, MODEL_NAME, version, exists):
        sys.exit(1)

    if not wait_until_ready(ENDPOINT_NAME):
        sys.exit(1)

    print("✅✅ PRODUCTION ENDPOINT DEPLOYMENT COMPLETED SUCCESSFULLY ✅✅")

    try:
        dbutils.notebook.exit("ENDPOINT_READY")
    except:
        pass
